In [1]:
import requests
import pandas as pd

url = 'https://api.apify.com/v2/datasets/tG0NVmk7nrsTOq39S/items'
response = requests.get(url)
data = response.json()

spouštět až odsud dolů (v případě chyb, testování...), ať pořád nevoláme api.

In [145]:
df = pd.DataFrame(data)

df.head(2)

,externalId,branchCode,name,distanceFromOrigin,address,coordinates,flags,openingHours,googleMapUrl,canBeSelected
0,I_12714,z-box-ceska-kubice-ceska-kubice-82,Z-BOX,100251.85,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'isNew': False, 'isClaimAssistant': False, 'i...","{'regular': {'monday': [{'open': '00:00', 'clo...",https://www.google.com/maps/dir/?api=1&destina...,True
1,I_12712,z-box-vseruby-u-kdyne-vseruby-40,Z-BOX,99734.36,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'isNew': False, 'isClaimAssistant': False, 'i...","{'regular': {'monday': [{'open': '00:00', 'clo...",https://www.google.com/maps/dir/?api=1&destina...,True


In [146]:
#Ve stávajícím DF vybrat sloupce, ostatní sloupce zahodit
df = df[['name', 'address', 'coordinates', 'openingHours']]

df.columns
df.head(2)

,name,address,coordinates,openingHours
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo..."


In [147]:
# Úprava sloupce address ve 4 krocích, ať je vidět postup.
df['address_text'] = df['address'].str.get('name')
# Vytáhnout hodnotu podle klíče "name" do proměnné "address_text"

df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Z-BOX Česká Kubice, Česká Kubice 82"
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Z-BOX Všeruby u Kdyně, Všeruby 40"


In [148]:
# Odstranit přesný text 'Z-Box' 
df['address_text'] = df['address_text'].str.replace('Z-BOX ', '', regex=False)

df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82"
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40"


Až po dokončení jsem zjistila (z .csv), že se sloupec adresy nerozdělil správně
Důvody 
- mezery na zač. a konci sloupce - odstraním, 
- závorky za č.p. - odstraním,
- v adresách jsou 2 a více čárek (původně jsem šla zleva po čárce a to bylo špatně)
    - nejprve ale odfiltruji adresy bez č.p.
    - potom vytvořím sloupec house_no dle mezery zprava (za kterou je č.p)
    - ze zbytku jdu zase zprava po první čárku.
  
Vracím se zpět do tohoto kroku a odstraním postupně tyto chyby. Tyto dodatečné 
úpravy nemusí být z kontextu jasné.

In [149]:
# Ořezat všechny mezery na začátku a konci
df['address_text'] = df['address_text'].str.strip()
df.columns
df.head()

,name,address,coordinates,openingHours,address_text
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82"
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40"
2,Z-BOX,"{'name': 'Z-BOX Nýrsko, Havlíčkova 474'}","{'latitude': 49.28962, 'longitude': 13.13596}","{'regular': {'monday': [{'open': '00:00', 'clo...","Nýrsko, Havlíčkova 474"
3,Z-BOX,"{'name': 'Z-BOX Hartmanice, Hartmanice 136'}","{'latitude': 49.1695, 'longitude': 13.45571}","{'regular': {'monday': [{'open': '00:00', 'clo...","Hartmanice, Hartmanice 136"
4,Z-BOX,"{'name': 'Z-BOX Kašperské Hory, Dlouhá 138'}","{'latitude': 49.14296, 'longitude': 13.5555}","{'regular': {'monday': [{'open': '00:00', 'clo...","Kašperské Hory, Dlouhá 138"


In [150]:
# Ukázka řádků, které koncí na (*)
adresy_se_zavorkou = df[df['address_text'].str.contains(r'\(.*\)$', na=False)]

df.columns
adresy_se_zavorkou

,name,address,coordinates,openingHours,address_text
9,Z-BOX,"{'name': 'Z-BOX Kdyně, Nádražní 747 (Tesco)'}","{'latitude': 49.38909, 'longitude': 13.03639}","{'regular': {'monday': [{'open': '00:00', 'clo...","Kdyně, Nádražní 747 (Tesco)"
10,Z-BOX,"{'name': 'Z-BOX Kdyně, Na Kobyle 209 (automyčk...","{'latitude': 49.39257, 'longitude': 13.02652}","{'regular': {'monday': [{'open': '00:00', 'clo...","Kdyně, Na Kobyle 209 (automyčka Comfort wapka)"
15,Z-BOX,"{'name': 'Z-BOX Volary, Soumarská (Stabe stave...","{'latitude': 48.91171, 'longitude': 13.8872}","{'regular': {'monday': [{'open': '00:00', 'clo...","Volary, Soumarská (Stabe stavebniny)"
17,Z-BOX,"{'name': 'Z-BOX Vimperk, Sušická 617 (Tesco)'}","{'latitude': 49.05857, 'longitude': 13.77849}","{'regular': {'monday': [{'open': '00:00', 'clo...","Vimperk, Sušická 617 (Tesco)"
21,Z-BOX,"{'name': 'Z-BOX Sušice, Hrádecká 1310 (Tesco)'}","{'latitude': 49.24116, 'longitude': 13.52214}","{'regular': {'monday': [{'open': '00:00', 'clo...","Sušice, Hrádecká 1310 (Tesco)"
...,...,...,...,...,...
5303,Z-BOX,"{'name': 'Z-BOX Rumburk, Pražská 1677 (Tesco)'}","{'latitude': 50.94735, 'longitude': 14.55785}","{'regular': {'monday': [{'open': '00:00', 'clo...","Rumburk, Pražská 1677 (Tesco)"
5310,Z-BOX,"{'name': 'Z-BOX Liberec, Hejnická 225 (Pro-Dom...","{'latitude': 50.80947, 'longitude': 15.03618}","{'regular': {'monday': [{'open': '00:00', 'clo...","Liberec, Hejnická 225 (Pro-Doma)"
5320,Z-BOX,"{'name': 'Z-BOX Šluknov, Sladovnická 1100 (Tes...","{'latitude': 51.00516, 'longitude': 14.454}","{'regular': {'monday': [{'open': '00:00', 'clo...","Šluknov, Sladovnická 1100 (Tesco)"
5325,Z-BOX,"{'name': 'Z-BOX Frýdlant, Fügnerova (automyčka...","{'latitude': 50.92302, 'longitude': 15.08627}","{'regular': {'monday': [{'open': '00:00', 'clo...","Frýdlant, Fügnerova (automyčka Jomi)"


In [151]:
# Odstranit závorku a mezeru před ní " (*)" z konce řádku.(Bez ?-závorka je daná)
df['address_text'] = df['address_text'].str.replace(r'\s*\(.*\)$', '', regex=True)

In [152]:
# Ukázka řádků, které koncí na (*)
adresy_se_zavorkou = df[df['address_text'].str.contains(r'\(.*\)$', na=False)]

df.columns
adresy_se_zavorkou

,name,address,coordinates,openingHours,address_text


In [153]:
#zobrazit adresy, kde jsou 2 a více čárek
adresy_vice_carek = df[df['address_text'].str.contains(r'(.*,){2,}', na=False)]

df.columns
adresy_vice_carek

C:\Users\Sabina\AppData\Local\Temp\ipykernel_20324\1016240191.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  adresy_vice_carek = df[df['address_text'].str.contains(r'(.*,){2,}', na=False)]


,name,address,coordinates,openingHours,address_text
196,Z-BOX,"{'name': 'Z-BOX Nová Bystřice, U Nádraží 381, ...","{'latitude': 49.01884, 'longitude': 15.1109}","{'regular': {'monday': [{'open': '00:00', 'clo...","Nová Bystřice, U Nádraží 381,"
244,Z-BOX,"{'name': 'Z-BOX Znojmo, Přímětice, Větrná 518/2'}","{'latitude': 48.87825, 'longitude': 16.03653}","{'regular': {'monday': [{'open': '00:00', 'clo...","Znojmo, Přímětice, Větrná 518/2"
343,Z-BOX,"{'name': 'Z-BOX Břeclav, Charvátská Nová Ves, ...","{'latitude': 48.76663, 'longitude': 16.8464}","{'regular': {'monday': [{'open': '00:00', 'clo...","Břeclav, Charvátská Nová Ves, Lednická 61/89"
853,Z-BOX,"{'name': 'Z-BOX Písek, Vinařického 374, 397 01'}","{'latitude': 49.31563, 'longitude': 14.14034}","{'regular': {'monday': [{'open': '00:00', 'clo...","Písek, Vinařického 374, 397 01"
880,Z-BOX,"{'name': 'Z-BOX Brno, Dolní Heršpice, Vídeňská...","{'latitude': 49.15491, 'longitude': 16.59786}","{'regular': {'monday': [{'open': '00:00', 'clo...","Brno, Dolní Heršpice, Vídeňská 132/100"
...,...,...,...,...,...
4728,Z-BOX,"{'name': 'Z-BOX Mladá Boleslav, Čejetice, Vodá...","{'latitude': 50.41302, 'longitude': 14.88569}","{'regular': {'monday': [{'open': '00:00', 'clo...","Mladá Boleslav, Čejetice, Vodárenská 162"
4776,Z-BOX,"{'name': 'Z-BOX Most, Souš, 28. října 108 (Smě...","{'latitude': 50.52493, 'longitude': 13.6283}","{'regular': {'monday': [{'open': '00:00', 'clo...","Most, Souš, 28. října 108"
5050,Z-BOX,"{'name': 'Z-BOX Teplice, Řetenice, Jateční 470'}","{'latitude': 50.64065, 'longitude': 13.80767}","{'regular': {'monday': [{'open': '00:00', 'clo...","Teplice, Řetenice, Jateční 470"
5206,Z-BOX,"{'name': 'Z-BOX Děčín, Boletice nad Labem, Stu...","{'latitude': 50.74126, 'longitude': 14.19233}","{'regular': {'monday': [{'open': '00:00', 'clo...","Děčín, Boletice nad Labem, Studniční 415"


In [154]:
# Ukázka řádků, kde není č.p
adresy_bez_number = df[df['address_text'].str.contains(r'\s+$', na=False)]

df.columns
adresy_bez_number

,name,address,coordinates,openingHours,address_text


In [155]:
# Ukázka řádků, kde nebyla čárka
df[df['address_text'].str.contains(',', na=False) == False]

df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82"
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40"


In [156]:
# Rozdělit řádek po 1. čárku (n=1), vytvořit nové sloupce (expand=True)
df[['city', 'street_and_number']] = df['address_text'].str.split(',', expand=True, n=1)

df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text,city,street_and_number
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82",Česká Kubice,Česká Kubice 82
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40",Všeruby u Kdyně,Všeruby 40


In [157]:
# Rozdělit ulici a číslo, po první mezeru zprava (rsplit)
if 'street_and_number' in df.columns:
    street_split = df['street_and_number'].str.rsplit(' ', n=1, expand=True)
    df['street'] = street_split[0]
    df['house_number'] = street_split[1]
    
df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text,city,street_and_number,street,house_number
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82",Česká Kubice,Česká Kubice 82,Česká Kubice,82
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40",Všeruby u Kdyně,Všeruby 40,Všeruby,40


In [158]:
# Vytáhnout hodnoty 'lat' a 'long'
df['latitude'] = df['coordinates'].str.get('latitude')
df['longitude'] = df['coordinates'].str.get('longitude')

df.columns
df.head(2)

,name,address,coordinates,openingHours,address_text,city,street_and_number,street,house_number,latitude,longitude
0,Z-BOX,"{'name': 'Z-BOX Česká Kubice, Česká Kubice 82'}","{'latitude': 49.36891, 'longitude': 12.8581}","{'regular': {'monday': [{'open': '00:00', 'clo...","Česká Kubice, Česká Kubice 82",Česká Kubice,Česká Kubice 82,Česká Kubice,82,49.36891,12.85810
1,Z-BOX,"{'name': 'Z-BOX Všeruby u Kdyně, Všeruby 40'}","{'latitude': 49.33973, 'longitude': 12.98162}","{'regular': {'monday': [{'open': '00:00', 'clo...","Všeruby u Kdyně, Všeruby 40",Všeruby u Kdyně,Všeruby 40,Všeruby,40,49.33973,12.98162


In [159]:
# Vymazat pomocné sloupce (axis - směr, horitontálně - 1)
df = df.drop(['address', 'address_text', 'street_and_number', 'coordinates'], axis=1)

df.columns
df.head(1)

,name,openingHours,city,street,house_number,latitude,longitude
0,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Česká Kubice,Česká Kubice,82,49.36891,12.8581


In [160]:
# Úprava sloupce openingHours - postupně rozbalím a uvidím, co s tím.
df['regular'] = df['openingHours'].str.get('regular')

df.columns
df.head(2)

,name,openingHours,city,street,house_number,latitude,longitude,regular
0,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Česká Kubice,Česká Kubice,82,49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'..."
1,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'..."


Tady jsem se blokla. Chtěla jsem řešit for cyklem, ale vůbec nevím, jak na to 
v Pandas

regular = {
    'monday': [{'open': '00:00', 'close': '23:59'}],
    'tuesday': [{'open': '00:00', 'close': '23:59'}],
    'wednesday': [{'open': '00:00', 'close': '23:59'}],
    'thursday': [{'open': '00:00', 'close': '23:59'}],
    'friday': [{'open': '00:00', 'close': '23:59'}],
    'saturday': [{'open': '00:00', 'close': '23:59'}],
    'sunday': [{'open': '00:00', 'close': '23:59'}],
    'exceptionDays': [
        {'from': '2025-04-18T00:00:00', 'times': [{'open': '00:00', 'close': '23:59'}]},
        {'from': '2025-04-21T00:00:00', 'times': [{'open': '00:00', 'close': '23:59'}]},
        ...
    ]
}
for den, seznam in regular.items():
    zaznam = seznam[0]
    for stav, cas in zaznam.items():
        print(den, stav, cas)

Toto se mi vypíše: to jsem si moc nepomohla.
monday open 00:00
monday close 23:59
tuesday open 00:00
tuesday close 23:59
wednesday open 00:00
wednesday close 23:59
thursday open 00:00
thursday close 23:59
friday open 00:00
friday close 23:59
saturday open 00:00
saturday close 23:59
sunday open 00:00
sunday close 23:59

Tak prostě loupu cibuli, nic jiného neumím.

In [161]:
# Úprava sloupce openingHours - postupně rozbalím a uvidím, co s tím, nic 
# lepšího neumím
df['monday'] = df['regular'].str.get('monday')
df['tuesday'] = df['regular'].str.get('tuesday')
df['wednesday'] = df['regular'].str.get('wednesday')
df['thursday'] = df['regular'].str.get('thursday')
df['friday'] = df['regular'].str.get('friday')
df['saturday'] = df['regular'].str.get('saturday')
df['sunday'] = df['regular'].str.get('sunday')

df.columns
df.head(2)

,name,openingHours,city,street,house_number,latitude,longitude,regular,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Česká Kubice,Česká Kubice,82,49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]"
1,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]"


Seznam slovníků - vybrat první pozici
Nakonec rozdělit dny na open and close a je to
na moji obranu, toto jsme fakt nebrali - aneb mám začátečnické znalosti
a nebojím se je využít

In [162]:
df['monday_0'] = df['monday'].str[0]
df['tuesday_0'] = df['tuesday'].str[0]
df['wednesday_0'] = df['wednesday'].str[0]
df['thursday_0'] = df['thursday'].str[0]
df['friday_0'] = df['friday'].str[0]
df['saturday_0'] = df['saturday'].str[0]
df['sunday_0'] = df['sunday'].str[0]

df.columns
df.head(2)


,name,openingHours,city,street,house_number,latitude,longitude,regular,monday,tuesday,...,friday,saturday,sunday,monday_0,tuesday_0,wednesday_0,thursday_0,friday_0,saturday_0,sunday_0
0,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Česká Kubice,Česká Kubice,82,49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]",...,"[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}"
1,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]",...,"[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}","{'open': '00:00', 'close': '23:59'}"


Zůstávají chybné sloupce i po restartu kernelu. Umím se jich zbavit jen 
selectem v dalším kroku

In [163]:
df['monday_open'] = df['monday_0'].str.get('open')
df['monday_close'] = df['monday_0'].str.get('close')

df['tuesday_open'] = df['tuesday_0'].str.get('open')
df['tuesday_close'] = df['tuesday_0'].str.get('close')

df['wednesday_open'] = df['wednesday_0'].str.get('open')
df['wednesday_close'] = df['wednesday_0'].str.get('close')

df['thursday_open'] = df['thursday_0'].str.get('open')
df['thursday_close'] = df['thursday_0'].str.get('close')

df['friday_open'] = df['friday_0'].str.get('open')
df['friday_close'] = df['friday_0'].str.get('close')

df['saturday_open'] = df['saturday_0'].str.get('open')
df['saturday_close'] = df['saturday_0'].str.get('close')

df['sunday_open'] = df['sunday_0'].str.get('open')
df['sunday_close'] = df['sunday_0'].str.get('close')

df.columns
df.head(2)

,name,openingHours,city,street,house_number,latitude,longitude,regular,monday,tuesday,...,wednesday_open,wednesday_close,thursday_open,thursday_close,friday_open,friday_close,saturday_open,saturday_close,sunday_open,sunday_close
0,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Česká Kubice,Česká Kubice,82,49.36891,12.85810,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]",...,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59
1,Z-BOX,"{'regular': {'monday': [{'open': '00:00', 'clo...",Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,"{'monday': [{'open': '00:00', 'close': '23:59'...","[{'open': '00:00', 'close': '23:59'}]","[{'open': '00:00', 'close': '23:59'}]",...,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59


Teď je ješě třeba projít finální scv a dočistit jednotlivé sloupce.
- house_number - přepsat vše co není číslo a nebo číslo/číslo - jako null

In [164]:
df = df[['name', 'city', 'street', 'house_number', 'latitude', 'longitude',	
         'monday_open', 'monday_close', 'tuesday_open', 'tuesday_close', 
         'wednesday_open', 'wednesday_close', 'thursday_open', 'thursday_close', 
         'friday_open', 'friday_close', 'saturday_open', 'saturday_close', 
         'sunday_open', 'sunday_close', 'openingHours']]

df.columns
df.head(2)

,name,city,street,house_number,latitude,longitude,monday_open,monday_close,tuesday_open,tuesday_close,...,wednesday_close,thursday_open,thursday_close,friday_open,friday_close,saturday_open,saturday_close,sunday_open,sunday_close,openingHours
0,Z-BOX,Česká Kubice,Česká Kubice,82,49.36891,12.85810,00:00,23:59,00:00,23:59,...,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,"{'regular': {'monday': [{'open': '00:00', 'clo..."
1,Z-BOX,Všeruby u Kdyně,Všeruby,40,49.33973,12.98162,00:00,23:59,00:00,23:59,...,23:59,00:00,23:59,00:00,23:59,00:00,23:59,00:00,23:59,"{'regular': {'monday': [{'open': '00:00', 'clo..."


Nespouštím poslední příkaz - uložení do json souboru, protože máv df pořád 
sloupec opening hours (nedohodly jsme se, co s tím.)


In [165]:
# import json

# df.to_json('zasilkovna_cisty_dataset.json', orient='records', force_ascii=False, indent=4)

In [166]:
df.to_csv('zasilkovna_cleaned_dataset.csv', index=False, encoding='utf-8')